# Cleaning a PostgreSQL Database

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

## Instructions

Let's dive into the world of SQL and work our magic to clean and analyze data from our hypothetical Super Store.

Find the top 5 products from each category based on highest total sales. The output should be sorted by category in ascending order and by sales in descending order within each category, i.e. within each category product with highest margin should sit on the top. Save the query as top_five_products_each_category, containing the following columns:

- category
- product_name
- product_total_sales (rounded to two decimal places)
- product_total_profit (rounded to two decimal places)
- product_rank

Calculate the quantity for orders with missing values in the quantity column. To do this, first calculate the unit price for each product based on the orders where the quantity is available, considering factors that might affect pricing. Then, use this unit price to estimate the missing quantity values for orders where the quantity is missing. The calculated values should be stored in the calculated_quantity column. Save query output as impute_missing_values, containing the following columns:

- product_id
- discount
- market
- region
- sales
- quantity
- calculated_quantity (rounded to zero decimal places)


In [ ]:
# Taking a quick look to the data
SELECT *
FROM orders
LIMIT 5

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,sales,quantity,discount,profit,shipping_cost,order_priority
0,957,MX-2014-105921,2014-05-28,2014-06-03,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,San Salvador,San Salvador,El Salvador,NaN,LATAM,Central,TEC-AC-10004626,342.080,2,0.00,0.0000,21.713,Medium
1,24359,ID-2013-61442,2013-01-15,2013-01-21,Standard Class,JB-16000,Joy Bell-,Consumer,Manila,National Capital,Philippines,NaN,APAC,Southeast Asia,OFF-BI-10001400,122.400,5,0.15,0.0000,21.710,Low
2,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024.0,US,East,TEC-AC-10003033,2309.650,7,0.00,762.1845,933.570,Critical
3,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,NaN,APAC,Oceania,FUR-CH-10003950,3709.395,9,0.10,-288.7650,923.630,Critical
4,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,NaN,APAC,Oceania,TEC-PH-10004664,5175.171,9,0.10,919.9710,915.490,Medium


In [4]:
SELECT *
FROM products
LIMIT 5

,product_id,category,sub_category,product_name
0,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...
1,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black"
2,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID"
3,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless"
4,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed"


In [ ]:
# Find the top 5 products from each category based on highest total sales. The output should be sorted by category in ascending order and by sales in descending order within each category
WITH ranked_products AS (
    SELECT 
        products.category,
        products.product_name,
        ROUND(SUM(ord.sales :: NUMERIC), 2) AS product_total_sales,
        ROUND(SUM(ord.profit :: NUMERIC), 2) AS product_total_profit,
        RANK() OVER(PARTITION BY products.category ORDER BY SUM(ord.sales) DESC) AS product_rank
    FROM orders AS ord
    INNER JOIN products ON ord.product_id = products.product_id
    GROUP BY products.category, products.product_name
)
SELECT * 
FROM ranked_products
WHERE product_rank < 6;

,category,product_name,product_total_sales,product_total_profit,product_rank
0,Furniture,"Hon Executive Leather Armchair, Adjustable",58193.48,5997.25,1
1,Furniture,"Office Star Executive Leather Armchair, Adjust...",51449.80,4925.80,2
2,Furniture,"Harbour Creations Executive Leather Armchair, ...",50121.52,10427.33,3
3,Furniture,"SAFCO Executive Leather Armchair, Black",41923.53,7154.28,4
4,Furniture,"Novimex Executive Leather Armchair, Adjustable",40585.13,5562.35,5
5,Office Supplies,"Eldon File Cart, Single Width",39873.23,5571.26,1
6,Office Supplies,"Hoover Stove, White",32842.60,-2180.63,2
7,Office Supplies,"Hoover Stove, Red",32644.13,11651.68,3
8,Office Supplies,"Rogers File Cart, Single Width",29558.82,2368.82,4
9,Office Supplies,"Smead Lockers, Industrial",28991.66,3630.44,5


In [ ]:
# Calculate the quantity for orders with missing values in the quantity column.
WITH missing_orders AS (
    SELECT 
        product_id, discount, market, region, sales, quantity
    FROM orders 
    WHERE quantity IS NULL
), 

unit_price_calculation AS (
    SELECT 
        product_id,
        AVG((sales / quantity) :: NUMERIC) AS unit_price
    FROM orders 
    WHERE quantity IS NOT NULL
    GROUP BY product_id
)

SELECT 
    mo.product_id, mo.discount, mo.market, mo.region, mo.sales, mo.quantity,
    ROUND(mo.sales:: NUMERIC / upc.unit_price, 0) AS calculated_quantity
FROM missing_orders AS mo
JOIN unit_price_calculation AS upc
    ON mo.product_id = upc.product_id

,product_id,discount,market,region,sales,quantity,calculated_quantity
0,TEC-STA-10003330,0.00,Africa,Africa,506.640,NaN,2
1,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,NaN,4
2,FUR-BO-10001337,0.15,US,West,308.499,NaN,3
3,TEC-STA-10004542,0.00,Africa,Africa,160.320,NaN,5
4,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,NaN,3
